In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import requests
import json
from pathlib import Path

In [15]:
notion_secret = "secret_5oM8WbfNkSB3SkfeZ38ncJNrpvVsttGrKbTqpDilTMW"
notion_database_id = "d4a484c209c0476692e4fbcc2ffe7d9f"

In [16]:
#Driver
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = options)

In [17]:
#Login page
login_url = "https://pe.computrabajo.com/login/"
driver.get(login_url)

In [18]:
#Login
input_email, *rest = driver.find_elements(By.CSS_SELECTOR, "#Email")
input_pass, *rest = driver.find_elements(By.CSS_SELECTOR, "#Password")

input_email.send_keys("info@japisale.com")
input_pass.send_keys("MilongoIV8")

btn, *rest = driver.find_elements(By.CSS_SELECTOR, "#bbR")
btn.click()

In [19]:
#Ir a Anuncio de empleo
driver.get("https://empresa.pe.computrabajo.com/Company/Offers/Match?oi=57776E486819C07761373E686DCF3405&cf=469814F59E4D6F04")

In [366]:
#Extraer los candidatos por página
people_list = driver.find_elements(By.CSS_SELECTOR, "article.rowuser.pos_rel.bClick.no_icons")

In [367]:
# Botón Siguiente en paginación
try:
    next_button = driver.find_element(By.CSS_SELECTOR, "div.paginas li.siguiente")
    print(next_button)
except:
    next_button = None

<selenium.webdriver.remote.webelement.WebElement (session="b123acf384b097fa68c23dda51658e1f", element="4F7268E8A7240A826CAE44099183C9C9_element_425")>


In [365]:
if next_button:
    next_button.click()

In [332]:
# Receorrer cada candidato y estraer información básica
#for people in pople_list:
# first = people_list[0]

In [333]:
def data_base_data(data):
    name, image, profile_page = data["name"], data["image"], data["profile_page"]
    grade, match, age, application_time = data["grade"], data["match"], data["age"], data["application_time"]
    
    notion_data = {
        "name": {
            "title": [{
                "text": {
                    "content": name
                }
            }]
        },
        "image": {
            "files": [
                {
                    "external": {
                        "url": image
                    },
                    "type": "external",
                    "name": name
                }
            ]
        },
        "profile_page": { "url": profile_page },
        "application_time": {
            "rich_text": [{
                "text": {
                    "content": application_time
                }
            }]
        },
        "age": { "number": age },
        "grade": {
            "rich_text": [{
                "text": {
                    "content": grade
                }
            }]
        },
        "match": { "number": match }
    }
    
    return notion_data

In [334]:
# Guardar en Notion los candidatos
async def send_to_notion(people_data):
    url = "https://api.notion.com/v1/pages"
    headers = dict([
        ("Authorization", f"Bearer {notion_secret}"),
        ("Content-Type", "application/json"),
        ("Notion-Version", "2022-06-28")
    ])

    data = dict([
        ("parent", { "database_id": notion_database_id }),
        ("properties", data_base_data(people_data))
    ])
    
    try:
        response, *rest = requests.post(url, headers=headers, data=json.dumps(data))
        return response
    except e:
        print("send_to_notion_error")
        print(e)
        return e

In [335]:
#people data
list_people = []
for people in people_list:

    try:
        name = people.find_element(By.CSS_SELECTOR, "ul li.nombre a").text
    except:
        name = ""

    try:
        image = people.find_element(By.CSS_SELECTOR, "ul li.nombre img.lazy").get_attribute("src")
    except:
        image = "??"

    try:
        profile_page = people.find_element(By.CSS_SELECTOR, "ul li.nombre a").get_attribute("href")
    except:
        profile_page = ""

    application_time  = people.find_element(By.CSS_SELECTOR, "ul li.aplicado").text
    age = people.find_element(By.CSS_SELECTOR, "ul li.edad").text
    grade = people.find_element(By.CSS_SELECTOR, "ul li.estudios").text
    match = people.find_element(By.CSS_SELECTOR, "ul li.adecuacion p").text
    people_data = dict([
        ("name", name),
        ("image", image),
        ("profile_page", profile_page),
        ("application_time", application_time),
        ("age", int(age)),
        ("grade", grade),
        ("match", int(match.split(" ")[0])),
    ])
    list_people.append(people_data)

In [336]:
len(list_people)

16

In [337]:
# Guardar en Notion
for p in list_people:
    try:
        res = await send_to_notion(p)
        print(res)
    except e:
        print(e)

b'{"object":"page","id":"6e842c8a-85a7-4f1c-8333-01bf80fc1532","created_time":"2023-10-02T01:35:00.000Z","last_edited_time":"2023-'
b'{"object":"page","id":"a3d7c249-8647-43d0-b814-645030146b17","created_time":"2023-10-02T01:35:00.000Z","last_edited_time":"2023-'
b'{"object":"page","id":"feb65cc2-cbcc-4bba-92e8-73044d1f992f","created_time":"2023-10-02T01:35:00.000Z","last_edited_time":"2023-'
b'{"object":"page","id":"af21d9f0-2d89-401b-910f-e0a55aaaa059","created_time":"2023-10-02T01:35:00.000Z","last_edited_time":"2023-'
b'{"object":"page","id":"fd2a9a51-2d29-4477-843d-3078c1a1f0d7","created_time":"2023-10-02T01:35:00.000Z","last_edited_time":"2023-'
b'{"object":"page","id":"a4d5b1e9-abc0-4f4a-8e5f-ef20d73483c9","created_time":"2023-10-02T01:35:00.000Z","last_edited_time":"2023-'
b'{"object":"page","id":"5b6e571e-6a62-4ba4-aee9-8d5f9a40ac3c","created_time":"2023-10-02T01:35:00.000Z","last_edited_time":"2023-'
b'{"object":"page","id":"b9d515cb-fd35-46a9-98da-94f2e43fc7de","created_time

In [23]:
li = driver.find_elements(By.CSS_SELECTOR, "ul.small.table li")

In [24]:
li[0].text

'anapausk12@gmail.com'

In [25]:
divs = driver.find_elements(By.CSS_SELECTOR, "div.bb1")

In [26]:
len(divs)

6

In [29]:
expe = divs[2]

In [31]:
lis = expe.find_elements(By.CSS_SELECTOR, "ul li")

In [32]:
lis[0].text

'Agosto de 2019 - Actualmente\n( 4 años y 2 meses )\nDiseñador grafico\nCamila Viali\nDescripción de funciones: -Creación, diseño gráfico y audiovisual de campañas digitales. -Identidad gráfica de la marca / Nuevas Colecciones / Cápsulas / Sub-Marcas. -Análisis de tendencias (benchmark). -Retoque fotográfico fashion. -Diseño y maquetación de Email marketing. -Diseño de catálogos. -Creación de landings page. Además de diseño y prototipado de interfaces (user flow, wireframes y testeo de usabilidad) para la página web CAMILA VIALI y AFFARI. -Optimización y gestión de las estrategias de marketing.'

In [34]:
a = {"a": 1}
b = { "b": 2 }
c = []

In [36]:
c.append({**a, **b})

In [37]:
c

[{'a': 1, 'b': 2}]

In [1]:
from pydantic import BaseModel
from typing import Union

In [10]:
class GHLContactModel(BaseModel):
    email: str
    source: str = "computrabajo_scrapper"

In [15]:
a = GHLContactModel(email="jj@mail.com")

In [24]:
a.model_dump_json()

'{"email":"jj@mail.com","source":"computrabajo_scrapper"}'